**To be covered**

1. Calculating loss using backprop algo, setting testing and val datasets
2. Pretraining and saving model weights
3. Loading Openai GPT2 weights into our architecture

![](images/eval-1.png)

![eval-2](images/eval-2.png)

**Generating Text Function**

1. unsqueeze(0) — Adding the "Batch" Dimension
Models like GPT are designed to process many sentences at once (a "batch") to be efficient. Because of this, they always expect a 2D or 3D input, even if you are only sending them a single sentence.

    Your input: [15496, 995] (Shape: [2]) — Just a simple list of words.

    Model expects: [[15496, 995]] (Shape: [1, 2]) — A batch containing one sentence.

    By calling .unsqueeze(0), you are telling PyTorch: "Add a new dimension at the very beginning (index 0) so this looks like a batch of 1."

2. squeeze(0) — Removing the "Batch" Dimension
Once the model is done and gives you an output, it still includes that extra batch "container." However, your tokenizer.decode() function doesn't know what a batch is—it just wants a flat list of numbers.

    Model output: tensor([[15496, 995, ...]]) (Shape: [1, 12])

    Decoder wants: [15496, 995, ...] (Shape: [12])

    By calling .squeeze(0), you are saying: "Take that outermost 'batch' dimension away so I can just get the list of word IDs back."

In [6]:
import tiktoken
import torch
from modules import generate_text_simple, GPTModel, GPT_CONFIG_124M

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flattened = token_ids.squeeze(0)
    return tokenizer.decode(flattened.tolist())

model = GPTModel(GPT_CONFIG_124M)
model.eval()  

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(model=model, idx=text_to_token_ids(start_context, tokenizer), max_new_tokens=10, context_size=GPT_CONFIG_124M["context_length"])

print("output text - ", token_ids_to_text(token_ids, tokenizer))

output text -  Every effort moves youlord Simone intu intro caloric vaccinationestablished ClassificationPortland oats


Gives gibberish as model is not yet trained, we need to train it and for that, we need to define an evalution metric, a framework which can let us know if the model is getting better, so lets define that loss